# Project #1: Tic-Tac-Toe

## Computers and Humans

Let's talk a little bit more about computers and humans and explore our relationships to each other.

Computers are:
- Dumb
- Fast
- Accurate

They're excellent at following instructions and doing *exactly* what we ask them to do in a timely manner. Computers will execute our instructions at blazing speeds and give us *immediate feedback* on our ideas. 

In comparison, humans are:
- Smart 
- Slow
- Prone to error

We're able to break down, figure out, and explain complex tasks, but need plenty of time to do so and communicate our thoughts. Even then, we're bound to make mistakes and struggle to communicate our ideas at times. Humans also have limited cognitive resources and can only spend so much time on a task. We tend to struggle with complexity and need to figure out ways to handle it. 

Wouldn't it be great if we could combine the best of both worlds? Just imagine it... if we can be smart, fast, and accurate with a computer to solve complex problems...

The good news is that we can! It just takes a little more effort upfront and a **commitment to transparent communication**.

This approach is **Test Driven Design**.

## Test Driven Design


## Tic-Tac-Toe Requirements

- R1: There will be two players, one human and one computer.
- R2: The only valid symbols allowed on the board are `X`, `O`, and `_`.
- R3: One player will be assigned `X` and the other, `O`.
- R4: The human player will choose to play `X` or `O` for the first game.
- R5: In each game afterwards, the most recent winner will choose to play `X` or `O`.
- R6: When a new game starts, all cells on the game board should be blank.
- R7: The player assigned `X` will make the first move.
- R8: After the first move, the players will alternate turns.
- R9: On one player's turn, that player will place their assigned symbol into a blank cell.
- R10: Once played, that `X` or `O` shall remain in that cell until the end of the game.
- R11: The first player to place three of their symbols in a row, column, or diagonal will be the winner.
- R12: When no blank squares remain and neither player has won, the game is a draw.

### STAGE 1
In this stage, your program should:

Take a string entered by the user and print out the game grid.

### STAGE 2
In this stage, your program should:

Take a string entered by the user and print the game grid as in the previous stage. Analyze the game state and print
the result. Possible states:
- "Game not finished" when neither side has three in a row but the grid still has empty cells.
- "Draw" when no side has a three in a row and the grid has no empty cells.
- "X" wins when the grid has three X’s in a row.
- "O" wins when the grid has three O’s in a row.
- "Impossible" when the grid has three X’s in a row as well as three O’s in a row, or there are a lot more X's than O's
  or vice versa (the difference should be 1 or 0; if the difference is 2 or more, then the game state is impossible).

In this stage, we will assume that either X or O can start the game.
You can choose whether to use a space or underscore _ to print empty cells.

### STAGE 3
The program should work as follows:

- Get the initial 3x3 grid from the input as in the previous stages. Here the user should input 9 symbols representing
  the field, for example, _XXOO_OX_.
- Output this 3x3 grid as in the previous stages.
- Prompt the user to make a move. The user should input 2 coordinate numbers that represent the cell where they want
  to place their X, for example, 1 1.
- Analyze user input. If the input is incorrect, inform the user why their input is wrong:
    - Print "This cell is occupied! Choose another one!" if the cell is not empty.
    - Print "You should enter numbers!" if the user enters non-numeric symbols in the coordinates input.
    - Print "Coordinates should be from 0 to 2!" if the user enters coordinates outside the game grid.
    - Keep prompting the user to enter the coordinates until the user input is valid.
- If the input is correct, update the grid to include the user's move and print the updated grid to the console.

To summarize, you need to output the field twice: 
- Once before the user's move (based on the first line of input)
- Once after the user has entered valid coordinates (then you need to update the grid to include that move).

### STAGE 4

In this stage, you should write a program that:

- Prints an empty grid at the beginning of the game.
- Creates a game loop where the program asks the user to enter the cell coordinates, analyzes the move for correctness
  and shows a grid with the changes if everything is okay.
- Ends the game when someone wins, if there is a draw, or if the human player signals they wish to exit.

In [124]:
from collections import namedtuple

Marker = namedtuple("Marker", ["symbol"])
X = Marker("X")
O = Marker("O")


class PositionOccupied(Exception):
    pass

class CannotMakeMove(Exception):
    pass


class GameBoard():
    
    __EMPTY = '-'
    
    __THREE_IN_A_ROW = ((0, 1, 2), (3, 4, 5), (6, 7, 8))
    __THREE_IN_A_COLUMN = ((0, 3, 6), (1, 4, 7), (2, 5, 8)) 
    __THREE_IN_A_DIAGONAL = ((0, 4, 8), (2, 4, 6))
    __WIN_CONDITIONS = __THREE_IN_A_ROW + __THREE_IN_A_COLUMN + __THREE_IN_A_DIAGONAL

    __demo_board = '''
   |   |   
 0 | 1 | 2 
___|___|___
   |   |   
 3 | 4 | 5 
___|___|___
   |   |   
 6 | 7 | 8 
   |   |   '''
    
        
    def __init__(self):
        self.board_map = self._blank_board()

    def display(self):
        print(f'''
   |   |   
 {self.board_map[0]} | {self.board_map[1]} | {self.board_map[2]} 
___|___|___
   |   |   
 {self.board_map[3]} | {self.board_map[4]} | {self.board_map[5]} 
___|___|___
   |   |   
 {self.board_map[6]} | {self.board_map[7]} | {self.board_map[8]} 
   |   |   ''')
        
    def display_demo_board(self):
        print(self.__demo_board)
    
    def register(self, marker: Marker, position: int):
        if position not in range(10):
            raise ValueError("position must be an integer from 0 to 9")            
        if marker.symbol not in ['X', 'O']:
            raise ValueError("marker must be either an 'X' or 'O' Marker")            
        if self.board_map.get(position) != self.__EMPTY:
            raise PositionOccupied("Cannot place game_piece in an occupied position")
        self.board_map[position] = marker.symbol
        
    def current_state(self) -> dict:
        return self.board_map
    
    def _filter_current_state_using(self, filter_function, dictionary) -> dict:
        return dict(filter(filter_function, dictionary.items()))
    
    def empty_spaces(self) -> dict:
        return self._filter_current_state_using(
            lambda items_pair: items_pair[1] == '-', 
            self.current_state()
        )
    
    def spaces_occupied_by(self, marker: Marker) -> dict:
        if marker.symbol not in ['X', 'O']:
            raise ValueError("marker must be either an 'X' or 'O' Marker")     
        return self._filter_current_state_using(
            lambda position_marker_pair: position_marker_pair[1] == marker.symbol, 
            self.current_state()
        )
    
    def reset(self):
        self.board_map = self._blank_board()
    
    def _blank_board(self) -> dict:
        return {cell: self.__EMPTY for cell in range(9)}
    
    def determine_if_game_has_ended(self) -> tuple:
        if self._win_conditions_reached_for(X):
            return (True, X)
        if self._win_conditions_reached_for(O):
            return (True, O)
        if not self.empty_spaces():
            return (True, None)
        return (False, None)
    
    def _win_conditions_reached_for(self, marker: Marker):
        currently_held_positions = self.spaces_occupied_by(marker)
        for win_condition in self.__WIN_CONDITIONS:
            matching_positions = 0
            for position in currently_held_positions:
                if position in win_condition:
                    matching_positions += 1
            if matching_positions == 3:
                return True
        return False

In [125]:
import random 


class Player():
    
    __admissable_positions = tuple(str(n) for n in range(9))
    
    
    def __init__(self, marker: Marker):
        self.marker = marker
        
    def current_marker(self) -> Marker:
        return self.marker
    
    def opposite_marker(self):
        return X if self.marker == O else O
    
    def set_marker(self, marker: Marker):
        self.marker = marker
        
    def place_marker(self, board) -> int:
        if not board.empty_spaces().keys():
            raise CannotMakeMove("An impossible state has been reached: The player has been prompted for a move, but there are no more playable moves.")
        human_player_input = input("Position of your next move: ")
        while not(human_player_input in self.__admissable_positions and int(human_player_input) in board.empty_spaces().keys()):
            print("Not an available position. Choose again.")
            human_player_input = input("Position of your next move: ")
        return int(human_player_input)
    
    def continue_playing(self) -> bool:
        human_player_input = input("Would you like to play again? Enter [y|n] ")
        while human_player_input not in ['y', 'n']:
            print("Please enter either [y|n]")
            human_player_input = input("Would you like to play again? Enter [y|n] ")
        return True if human_player_input == 'y' else False
    
    def choose_next_marker(self):
        human_player_input = input("Winner's choice: 'X' or 'O'? ")
        while human_player_input not in ['X', 'O']:
            print("Please enter either 'X' or 'O'")
            human_player_input = input("Your choice: 'X' or 'O'? ")
        self.set_marker(X if human_player_input == 'X' else O)    
    
    
class RobotPlayer():
    
    def __init__(self, marker: Marker):
        self.marker = marker
        
    def current_marker(self) -> Marker:
        return self.marker
    
    def opposite_marker(self):
        return X if self.marker == O else O
    
    def set_marker(self, marker: Marker):
        self.marker = marker
        
    def place_marker(self, board) -> int:
        available_positions = board.empty_spaces().keys()
        if not available_positions:
            raise CannotMakeMove("An impossible state has been reached: There are no available choices for the robot to make.") 
        available_indexable_positions = tuple(available_positions)
        choice = random.choice(available_indexable_positions)
        print(f"Robot selected position: {choice}")
        return choice
    
    def choose_next_marker(self):
        if self.current_marker() != X:
            self.set_marker(self.opposite_marker())
        print(f"\nRobot is {self.current_marker().symbol}")

In [126]:
class TicTacToeApp():
    
    __LOGO = """
▀█▀ █ █▀▀ ▀█▀ ▄▀█ █▀▀ ▀█▀ █▀█ █▀▀
░█░ █ █▄▄ ░█░ █▀█ █▄▄ ░█░ █▄█ ██▄
"""
    
    
    def __init__(self):
        print(self.__LOGO)
        self.continue_playing = True
        self.board = GameBoard()
        self.board.display_demo_board()
        self.human_player = Player(self._players_initial_choice())
        self.robot_player = RobotPlayer(self.human_player.opposite_marker())

    def launch(self):
        while self.continue_playing:
            game_over = False
            winner = None
            while not game_over:
                if self.human_player.current_marker() == X:
                    self.board.display()
                    self.board.register(X, self.human_player.place_marker(self.board))
                    game_over, winner = self.board.determine_if_game_has_ended()
                    if game_over:
                        break
                    self.board.register(O, self.robot_player.place_marker(self.board))
                    game_over, winner = self.board.determine_if_game_has_ended()
                    if game_over:
                        break
                else:
                    self.board.register(X, self.robot_player.place_marker(self.board))
                    game_over, winner = self.board.determine_if_game_has_ended()
                    if game_over:
                        break
                    self.board.display()
                    self.board.register(O, self.human_player.place_marker(self.board))
                    game_over, winner = self.board.determine_if_game_has_ended()
                    if game_over:
                        break
            self._declare_game_results(winner)
            self.continue_playing = self.human_player.continue_playing()
            if self.continue_playing:
                self._setup_new_game(winner)
        print("Thank you for playing.")
        
    def _players_initial_choice(self) -> Marker:
        human_player_input = input("Your choice: 'X' or 'O'? ")
        while human_player_input not in ['X', 'O']:
            print("Please enter either 'X' or 'O'")
            human_player_input = input("Your choice: 'X' or 'O'? ")
        return X if human_player_input == 'X' else O
    
    def _declare_game_results(self, winner):
        self.board.display()
        if not winner:
            print("The game ended in a draw" )
        else:
            print(f"The game has ended. {winner.symbol} has won.")
    
    def _setup_new_game(self, winner):
        self.board.reset()
        if self.human_player.current_marker() == winner:
            self.human_player.choose_next_marker()
            self.robot_player.set_marker(self.human_player.opposite_marker())
        elif self.robot_player.current_marker() == winner:
            self.robot_player.choose_next_marker()
            self.human_player.set_marker(self.robot_player.opposite_marker())
        else:
            print("After a draw both players will keep their current markers.")
        print(f"Player is {self.human_player.current_marker().symbol}")

In [127]:
app = TicTacToeApp()
app.launch()


▀█▀ █ █▀▀ ▀█▀ ▄▀█ █▀▀ ▀█▀ █▀█ █▀▀
░█░ █ █▄▄ ░█░ █▀█ █▄▄ ░█░ █▄█ ██▄


   |   |   
 0 | 1 | 2 
___|___|___
   |   |   
 3 | 4 | 5 
___|___|___
   |   |   
 6 | 7 | 8 
   |   |   
Your choice: 'X' or 'O'? X

   |   |   
 - | - | - 
___|___|___
   |   |   
 - | - | - 
___|___|___
   |   |   
 - | - | - 
   |   |   
Position of your next move: 1
Robot selected position: 2

   |   |   
 - | X | O 
___|___|___
   |   |   
 - | - | - 
___|___|___
   |   |   
 - | - | - 
   |   |   
Position of your next move: 4
Robot selected position: 3

   |   |   
 - | X | O 
___|___|___
   |   |   
 O | X | - 
___|___|___
   |   |   
 - | - | - 
   |   |   
Position of your next move: 5
Robot selected position: 8

   |   |   
 - | X | O 
___|___|___
   |   |   
 O | X | X 
___|___|___
   |   |   
 - | - | O 
   |   |   
Position of your next move: 6
Robot selected position: 0

   |   |   
 O | X | O 
___|___|___
   |   |   
 O | X | X 
___|___|___
   |   |   
 X | - | O 
   |   |   
Position of your next